## Dữ liệu Streaming

In [ ]:
# pip install kafka-python websocket-client python-dotenv
import os
import json
import time
from datetime import datetime, timezone

from dotenv import load_dotenv
import websocket

from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError, NoBrokersAvailable

# ====== ENV & cấu hình ======
load_dotenv()

TOKEN = os.getenv("FINNHUB_API_TOKEN")
if not TOKEN:
    raise RuntimeError("Thiếu FINNHUB_API_TOKEN trong .env")

# Kết nối WS Finnhub
WS_URL = f"wss://ws.finnhub.io?token={TOKEN}"

# Kafka
KAFKA_BOOTSTRAP = os.getenv("KAFKA_BOOTSTRAP", "localhost:9092")  # host: dùng localhost:9092
KAFKA_TOPIC     = os.getenv("KAFKA_TOPIC", "crypto.trades")       # tên topic
KAFKA_PARTITIONS = int(os.getenv("KAFKA_PARTITIONS", "6"))
KAFKA_RF         = int(os.getenv("KAFKA_RF", "1"))                # 1 cho cluster đơn node

# WS keepalive
PING_INTERVAL = 15
PING_TIMEOUT  = 10
SUB_DELAY     = 0.5  # nương tay khi subscribe

SYMBOLS = [
    "BINANCE:BTCUSDT",
    "BINANCE:ETHUSDT",
    "BINANCE:BNBUSDT",
]

# ====== Kafka utils ======
def ensure_topic(bootstrap: str, topic: str, partitions: int = 6, rf: int = 1, configs: dict | None = None):
    """
    Kiểm tra topic; nếu chưa có thì tạo mới.
    Trả về True nếu OK, raise nếu không kết nối được broker.
    """
    admin = None
    try:
        admin = KafkaAdminClient(bootstrap_servers=bootstrap, client_id="topic-manager")
        # Kiểm tra tồn tại
        existing = admin.list_topics()
        if topic in existing:
            return True
        # Tạo mới
        nt = NewTopic(name=topic, num_partitions=partitions, replication_factor=rf, topic_configs=configs or {})
        admin.create_topics([nt], validate_only=False)
        return True
    except TopicAlreadyExistsError:
        return True
    except NoBrokersAvailable as e:
        raise RuntimeError(f"Không kết nối được Kafka tại {bootstrap}: {e}") from e
    finally:
        if admin:
            admin.close()

def build_producer(bootstrap: str) -> KafkaProducer:
    """
    Tạo KafkaProducer với cấu hình hợp lý cho stream trade.
    """
    return KafkaProducer(
        bootstrap_servers=bootstrap,
        acks="all",                 # an toàn hơn (leader + ISR)
        linger_ms=50,               # gộp lô nhẹ
        batch_size=64 * 1024,       # 64KB
        retries=5,
        value_serializer=lambda v: json.dumps(v, ensure_ascii=False).encode("utf-8"),
        key_serializer=lambda k: k.encode("utf-8") if k else None,
    )

# ====== Helpers thời gian ======
def now_utc():
    return datetime.now(timezone.utc)

def epoch_ms(dt: datetime) -> int:
    return int(dt.timestamp() * 1000)

# ====== Kafka producer (global) ======
producer: KafkaProducer | None = None

def send_to_kafka(rec: dict):
    """
    Produce 1 record vào Kafka (key = symbol, value = JSON).
    Dùng trade_ts (nếu có) làm timestamp của message.
    """
    global producer
    if producer is None:
        return
    try:
        key = rec.get("symbol")
        ts_ms = rec.get("trade_ts") or rec.get("recv_ts")
        # Gửi, không flush mỗi bản ghi (flush khi shutdown)
        producer.send(KAFKA_TOPIC, key=key, value=rec, timestamp_ms=int(ts_ms) if ts_ms else None)
    except Exception as e:
        print("Kafka produce error:", e)

def close_kafka():
    global producer
    if producer:
        try:
            producer.flush(timeout=5)
            producer.close(timeout=5)
        except Exception:
            pass
        producer = None

# ====== WS callbacks ======
def on_message(ws, message):
    recv_dt = now_utc()
    recv_ms = epoch_ms(recv_dt)

    try:
        msg = json.loads(message)
    except Exception:
        return

    if not isinstance(msg, dict) or msg.get("type") != "trade":
        return

    data = msg.get("data") or []
    for d in data:
        symbol = d.get("s")
        price  = d.get("p")
        volume = d.get("v")
        trade_ts = d.get("t")  # epoch ms
        cond   = d.get("c")

        # Fallback timestamp nếu thiếu t
        trade_ms = trade_ts if isinstance(trade_ts, int) else recv_ms
        trade_dt = datetime.fromtimestamp(trade_ms / 1000, tz=timezone.utc)

        rec = {
            "source": "finnhub",
            "type": "trade",
            "symbol": symbol,
            "price": price,
            "volume": volume,
            "trade_ts": trade_ms,
            "recv_ts": recv_ms,
            "conditions": cond,
            "trade_time_iso": trade_dt.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z",
        }

        # ĐẨY VÀO KAFKA
        send_to_kafka(rec)

def on_error(ws, error):
    print("WS error:", error)

def on_close(ws, code, msg):
    print("WS closed", code, msg)

def on_open(ws):
    # Đăng ký 10 symbol, “nương tay” để tránh throttle
    for sym in SYMBOLS:
        ws.send(json.dumps({"type": "subscribe", "symbol": sym}))
        time.sleep(SUB_DELAY)
    print(f"Subscribed {len(SYMBOLS)} symbols.")

# ====== Main loop ======
def run():
    # 1) Đảm bảo topic tồn tại
    ensure_topic(KAFKA_BOOTSTRAP, KAFKA_TOPIC, partitions=KAFKA_PARTITIONS, rf=KAFKA_RF)

    # 2) Tạo producer
    global producer
    producer = build_producer(KAFKA_BOOTSTRAP)

    # 3) Giữ kết nối WS + reconnect with backoff
    backoff = 1
    while True:
        try:
            ws = websocket.WebSocketApp(
                WS_URL,
                on_open=on_open,
                on_message=on_message,
                on_error=on_error,
                on_close=on_close,
            )
            ws.run_forever(ping_interval=PING_INTERVAL, ping_timeout=PING_TIMEOUT)
        except KeyboardInterrupt:
            print("Stopping by user...")
            break
        except Exception as e:
            print("WS exception:", e)

        # Backoff khi đứt kết nối
        time.sleep(backoff)
        backoff = min(backoff * 2, 60)

    # 4) Đóng Kafka producer gọn
    close_kafka()

if __name__ == "__main__":
    run()


Subscribed 3 symbols.


## Dữ liệu Batch

In [7]:
from vnstock import Listing
listing = Listing()

vnstock_list = listing.all_symbols()['symbol'].to_list()

In [22]:

import pandas as pd
from vnstock import Quote, Listing
from datetime import datetime, timezone, timedelta
import time # Thêm thư viện time để tạm dừng

# --- Cài đặt thời gian ---
vn_tz = timezone(timedelta(hours=7))
now_vn = datetime.now(vn_tz)
today_str = now_vn.strftime("%Y-%m-%d")
START_DATE = '2025-01-01'
RESOLUTION = '1D' # BẮT BUỘC dùng '1D' cho lịch sử dài hạn
DELAY_SECONDS = 0.5 # Thêm độ trễ để tránh bị khóa IP

# --- Bước 1: Khởi tạo Listing và lấy danh sách mã ---
try:
    listing_obj = Listing() # Sửa lỗi 1: Khởi tạo đối tượng
    df_symbols = listing_obj.all_symbols()
    vnstock_list = df_symbols['symbol'].to_list()
    print(f"Tìm thấy {len(vnstock_list)} mã. Bắt đầu tải...")
except Exception as e:
    print(f"Không thể lấy danh sách mã: {e}")
    vnstock_list = [] # Gán list rỗng để code không chạy tiếp

# --- Bước 2: Chuẩn bị list để thu thập ---
list_of_dfs = []
symbol_no_data = [] # Danh sách các mã không tải được hoặc không có dữ liệu

# --- Bước 3: Vòng lặp ---
for i, symbol in enumerate(vnstock_list):
    # Thêm (i+1) để biết đang chạy đến đâu
    print(f"({i+1}/{len(vnstock_list)}) Đang tải: {symbol}...")
    
    try:
        quote = Quote(symbol=symbol, source='VCI')
        # Sửa lỗi 2: Dùng RESOLUTION ('1D') thay vì '1m'
        data = quote.history(start=START_DATE, end=today_str, interval=RESOLUTION)
        
        # Kiểm tra xem có dữ liệu trả về không
        if not data.empty:
            # Sửa lỗi 4: Thêm cột 'ticker' để nhận diện
            data['ticker'] = symbol 
            list_of_dfs.append(data)
        else:
            # Nếu data rỗng (mã mới, không có giao dịch...)
            print(f"  -> Không có dữ liệu cho {symbol} trong khoảng thời gian này.")
            symbol_no_data.append(symbol)

    except Exception as e:
        # Nếu API báo lỗi (mã không tìm thấy, lỗi mạng...)
        print(f"  -> Lỗi khi tải {symbol}: {e}")
        # Sửa lỗi 3: Dùng .append() đúng cách
        symbol_no_data.append(symbol) 
    
    # Thêm độ trễ để tránh làm quá tải server API
    time.sleep(DELAY_SECONDS)

# --- Bước 4: Tổng hợp dữ liệu ---
if list_of_dfs:
    print("\nĐang tổng hợp dữ liệu...")
    df_tong = pd.concat(list_of_dfs, ignore_index=True)
    
    print("--- HOÀN TẤT ---")
    print("DataFrame tổng (5 dòng đầu):")
    print(df_tong.head())
    print("\nDataFrame tổng (5 dòng cuối):")
    print(df_tong.tail())
else:
    print("\nKhông tải được dữ liệu nào, DataFrame tổng bị rỗng.")

print(f"\nCác mã không có dữ liệu hoặc bị lỗi: {len(symbol_no_data)}")
print(symbol_no_data)

Tìm thấy 1721 mã. Bắt đầu tải...
(1/1721) Đang tải: YTC...
(2/1721) Đang tải: YEG...
(3/1721) Đang tải: YBM...
(4/1721) Đang tải: YBC...
(5/1721) Đang tải: XPH...
(6/1721) Đang tải: XMP...
(7/1721) Đang tải: XMD...
(8/1721) Đang tải: XMC...
(9/1721) Đang tải: XLV...
(10/1721) Đang tải: XHC...
(11/1721) Đang tải: XDH...
(12/1721) Đang tải: XDC...
  -> Lỗi khi tải XDC: RetryError[<Future at 0x22492534690 state=finished raised ValueError>]
(13/1721) Đang tải: X77...
  -> Lỗi khi tải X77: RetryError[<Future at 0x22492978450 state=finished raised ValueError>]
(14/1721) Đang tải: X26...
(15/1721) Đang tải: X20...
(16/1721) Đang tải: WTC...
(17/1721) Đang tải: WSS...
(18/1721) Đang tải: WSB...
(19/1721) Đang tải: WCS...
(20/1721) Đang tải: VXT...
(21/1721) Đang tải: VXP...
  -> Lỗi khi tải VXP: RetryError[<Future at 0x2248f07e910 state=finished raised ValueError>]
(22/1721) Đang tải: VXB...
(23/1721) Đang tải: VWS...
(24/1721) Đang tải: VW3...
(25/1721) Đang tải: VVS...
(26/1721) Đang tải: VV

In [26]:
df_tong.to_csv('./data/vnstock/vnstock_1d.csv', index=False, encoding='utf-8-sig')